# CityBikes Rest API

This section asks you to fetch JSON data from the [CityBikes REST API](https://api.citybik.es/v2/) to answer some questions about bike share programs.

Please be mindful not to send requests to the server more often than necessary. In particular, make sure you separate code that requests data from the server into a separate cell so that you do not have to repeatedly request data from the server. You will lose points if your requests are not separated into their own cell.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
import pandas as pd
from pandas.io.json import json_normalize

In [0]:
import requests
response = requests.get("http://api.citybik.es/v2/networks")

In [0]:
data = json.loads(response.text)
df = json_normalize(data, "networks")

In [0]:
df

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN
1,[Gobike A/S],/v2/networks/bycyklen,bycyklen,Bycyklen,Copenhagen,DK,55.673582,12.564984,NaN,NaN,NaN,NaN
2,[Gobike A/S],/v2/networks/nu-connect,nu-connect,Nu-Connect,Utrecht,NL,52.117000,5.067000,NaN,NaN,NaN,NaN
3,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN
4,[Gobike A/S],/v2/networks/bysykkelen,bysykkelen,Bysykkelen,Stavanger,NO,58.969975,5.733107,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
640,[Nextbike GmbH],/v2/networks/nextbike-odesa,nextbike-odesa,Nextbike,Odesa,UA,46.467200,30.716400,NaN,NaN,NaN,NaN
641,[Nextbike GmbH],/v2/networks/nextbike-alvsjo,nextbike-alvsjo,Nextbike,Älvsjö,DE,59.274500,18.005000,NaN,NaN,NaN,NaN
642,[Nextbike GmbH],/v2/networks/nextbike-wittenberge,nextbike-wittenberge,Nextbike,Wittenberge,DE,52.995200,11.751700,NaN,NaN,NaN,NaN
643,[Nextbike GmbH],/v2/networks/frelo-freiburg,frelo-freiburg,Frelo,Freiburg,DE,47.995800,7.844530,NaN,NaN,NaN,NaN


# Question 1

Find all cities with bike share programs in the United States (country code "US"), along with their network ID (for example, "ford-gobike"). How many cities in the U.S. have bike share programs (at least that are in this API)?

In [0]:
q1.head()

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url
75,"[PBSC, Alta Bicycle Share, Inc]",/v2/networks/we-cycle,we-cycle,WE-cycle,"Aspen, CO",US,39.194951,-106.837002,NaN,https://asp.publicbikesystem.net/ube/gbfs/v1/g...,NaN,NaN
132,"[Clean Energy Coalition, BCycle, LLC]",/v2/networks/arborbike,arborbike,ArborBike,"Ann Arbor, MI",US,42.278530,-83.745360,NaN,https://gbfs.bcycle.com/bcycle_arborbike/gbfs....,NaN,NaN
135,"[BCycle, LLC]",/v2/networks/austin,austin,Austin B-cycle,"Austin, TX",US,30.264080,-97.743550,NaN,https://gbfs.bcycle.com/bcycle_austin/gbfs.json,NaN,NaN
136,"[Motivate International, Inc, PBSC]",/v2/networks/bike-chattanooga,bike-chattanooga,Bike Chattanooga,"Chattanooga, TN",US,35.045630,-85.309680,NaN,https://chat.publicbikesystem.net/ube/gbfs/v1/,NaN,NaN
139,"[Portland Bureau of Transportation (PBOT), Mot...",/v2/networks/biketown,biketown,BIKETOWN,"Portland, OR",US,45.521754,-122.681079,NaN,http://biketownpdx.socialbicycles.com/opendata...,NaN,NaN


In [0]:
q1 = df[df["location.country"] == "US"]
q1["location.city"].nunique()

64

64 cities have bike share programs in the US 

# Question 2

Construct a `DataFrame` containing data about all bike stations in all networks in the United States. Save this `DataFrame` to disk using `.to_csv()`. (You will need it in Part B of this lab.)

Use this `DataFrame` to determine the total number of bicycles in bike share programs across the United States? You may assume that the number of bikes at a station is the number of empty spaces, plus the number of available bikes.

In [0]:
url = "http://api.citybik.es/v2/networks/network_id"

In [0]:
response = requests.get(url)

In [0]:
import time

us = []
for city_id in q1["id"]:
    
    # get the episodes for the show from the REST API
    response = requests.get("http://api.citybik.es/v2/networks/%s" % city_id)
    us.append(response.json())
    
    # add a 0.5 second delay between each query
    time.sleep(0.5)
    
# Now we have a list of episodes in JSON format.
# We can convert this to a DataFrame of episodes using json_normalize.

In [0]:
df_2 = json_normalize([item['network'] for item in us], 'stations', meta='id', meta_prefix='network', errors="ignore")
df_2

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,extra.bike_uids,extra.number,extra.slots,networkid
0,2.0,3,de99fa376a6ff38db656affe33d40e52,39.380066,-107.081968,Evans Rd,2020-03-02T05:22:32.985000Z,Basalt,1.572298e+09,1.0,1.0,1482,NaN,NaN,NaN,we-cycle
1,8.0,0,c7fddca9c7f33987a59095d9fa1dba04,39.191600,-106.823000,Hotel Aspen,2020-03-02T05:22:32.995000Z,Aspen,1.572470e+09,0.0,1.0,1480,NaN,NaN,NaN,we-cycle
2,6.0,0,edd6bdd575f305389a942010c19b3fd0,39.359289,-107.023232,Roaring Fork Club Employee Housing,2020-03-02T05:22:32.996000Z,Basalt,1.572385e+09,0.0,1.0,1485,NaN,NaN,NaN,we-cycle
3,7.0,0,b7130e6c2093f16d6462e6c778db62f4,39.188670,-106.850827,Aspen Recreation Center,2020-03-02T05:22:32.997000Z,Aspen,1.572464e+09,0.0,1.0,1443,NaN,NaN,NaN,we-cycle
4,10.0,1,041409c38a1cc57e8a0c49b5ef8458fe,39.394798,-107.095600,Sopris View,2020-03-02T05:22:32.999000Z,Basalt,1.572378e+09,0.0,1.0,1421,NaN,NaN,NaN,we-cycle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5668,3.0,3,a7a14d5f2cb56bae5aa825f88f13e0a9,40.771747,-74.038253,Applied Housing,2020-03-02T05:22:55.461000Z,NaN,NaN,NaN,NaN,8650453,"[73213, 73279, 73776]",1989,6.0,jerseybike-north-bergen
5669,0.0,4,36c6627d7ad58656abf75f9ab13c6a27,40.797553,-73.998183,Braddock Park South,2020-03-02T05:22:55.462000Z,NaN,NaN,NaN,NaN,8650478,"[73563, 73409, 73760, 73637]",1985,4.0,jerseybike-north-bergen
5670,3.0,3,a7c41e6021ca966e92e620ae765a4054,40.809077,-74.001292,Braddock Park North,2020-03-02T05:22:55.463000Z,NaN,NaN,NaN,NaN,8650486,"[73219, 73755, 73641]",1986,6.0,jerseybike-north-bergen
5671,NaN,8,84a596a3e5879bc668deb1f848b9d353,40.707994,-74.035445,Liberty State Park North---Central Railroad (V...,2020-03-02T05:24:47.960000Z,NaN,NaN,NaN,NaN,449810,"[73222, 73196, 73169, 73305, 73278, 73260, 737...",1934,NaN,jerseybike-liberty-state-park


In [0]:
us[0]

In [0]:
df_2.to_csv('drive/My Drive/dataframe.csv')

# Question 3

You have just finished touring Coit Tower in San Francisco, which is located at latitude 37.802747 and longitude -122.405861. Using your `DataFrame` from Question 2, find the nearest bike station with an available bike, based on taxicab distance (a.k.a. Manhattan distance).

_Hint:_ You can check your answer using Google Maps!

In [0]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(df_2)
# df_2_st = scaler.transform(df_2)

lats = df_2.latitude
longs = df_2.longitude

total1 = ((lats - 37.803).abs())
total2 = ((longs - -122.405).abs())
total = total1+total2

In [0]:
total.idxmin()

4823

In [0]:
df_2.iloc[4823]

empty_slots                                          5
free_bikes                                          17
id                    28754859653e0a656ba53e1fa0ea21db
latitude                                       37.8048
longitude                                     -122.403
name                     The Embarcadero at Sansome St
timestamp                  2020-03-02T05:26:22.019000Z
extra.address                                     None
extra.last_updated                         1.58313e+09
extra.renting                                        1
extra.returning                                      1
extra.uid                                            6
extra.bike_uids                                    NaN
extra.number                                       NaN
extra.slots                                        NaN
networkid                                  ford-gobike
Name: 4823, dtype: object

## Submission Instructions

- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 5A - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you and your partner (if you worked with one).
- Share the notebook with `data301labs@gmail.com`, giving commenting rights.
  - Click on "Share" at the top-right. 
  - Type in the `data301labs@gmail.com` into the box that appears. 
  - Click on the pencil at the right-hand side and change it from "Can edit" to "Can comment". 
  - Make sure the "Notify people" box in the bottom right is checked. 
  - Click the "Done" button.